## 导入必要的库函数

In [7]:
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

%matplotlib qt

In [8]:
model_accuracies = {
    'lennet_5': []
}
print(model_accuracies)

{'lennet_5': []}


## 深度学习模型一 lennet-5

### 1.图像转化

In [9]:
# 定义批量大小
batch_size = 100
# 数据增强和预处理
transform1 = transforms.Compose([
    transforms.Resize((32, 32)),##图像大小转化
    transforms.ToTensor(),## 将图像转换为PyTorch的张量格式，并将像素值从0-255缩放到0-1之间
   transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])## 标准化图像，使每个通道的像素值分布在[-1, 1]之间。这里，均值和标准差为[0.485, 0.456, 0.406]和。[0.229, 0.224, 0.225]，计算ImageNet数据集所有图像的颜色通道均值和标准差得到的。
])

### 2.载入图片数据

In [10]:
# 加载数据集
data_dir = r'D:\深度学习练习\Brain tumor classification-20240725-1.0(1)'
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform1)#数据转化
train_size = int(0.8 * len(train_dataset))#数据集划分，2：8划分，确定训练集大小
val_size = len(train_dataset) - train_size#确定测试集大小
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])#随机从数据集中分配数据到测试，训练集

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)#数据分批载入训练集，每批次大小为batch_size,shuffle=True在每个epoch开始时，数据集中的样本会被随机重排
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### 3.定义模型1，lenet-5

In [11]:
# 定义 LeNet-5 模型
class LeNet5(nn.Module):
    def __init__(self, num_classes=4):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)  # 3个通道输入输出6，第一层卷积窗口大小为5，步长为2
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)#第一层池化，使用平均池化，窗口大小为2，步长为2，把卷积后的图片放小为原理的四分之一
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 调整展平后的大小
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)  # 输出5类

    def forward(self, x):#向前传播更新
        x = torch.relu(self.conv1(x))
        #print(f"out shape1: {x.shape}")
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        #print(f"out shape2: {x.shape}")
        x = self.pool2(x)
        #print(f"out shape3: {x.shape}")
        x = x.view(-1, 16 * 6 * 6)  # 确保形状匹配
        #print(f"out shape4: {x.shape}")
        x = torch.relu(self.fc1(x))
        #print(f"out shape5: {x.shape}")
        x = torch.relu(self.fc2(x))
        #print(f"out shape6: {x.shape}")
        x = self.fc3(x)
        #print(f"out shape7: {x.shape}")
        return x

# 参数设置
learning_rate = 0.0001#学习效率
epochs = 40#训练次数
# 检查GPU是否可用，并把模型部署到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### 4.初始化训练模型1

In [12]:
# 初始化模型、损失函数和优化器
model = LeNet5(num_classes=4).to(device)
criterion = nn.CrossEntropyLoss()#选择CrossEntropyLoss为损失函数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)# 优化器选择为Adam
#效果很差criterion = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
accuracies = []
# 训练模型
train_losses = []
val_losses = []

for epoch in range(epochs):
    model.train()#将模型设置为训练模式
    running_loss = 0.0# 初始化一个变量来累积每个epoch的损失
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)#将训练数据部署到GPU上
        optimizer.zero_grad()# 清除优化器的梯度
        outputs = model(inputs) # 前向传播：使用模型预测输出
        loss = criterion(outputs, labels)#计算损失
        loss.backward()# 反向传播：计算梯度
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(train_loader))
    print(f"Epoch {epoch+1}, Train Loss: {running_loss / len(train_loader)}")
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到GPU
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    # 将准确率存储到 model_accuracies 字典中
    model_accuracies['lennet_5'].append(accuracy)
    print(f'Epoch [{epoch+1}/{epochs}], Validation Accuracy: {accuracy:.2f}%')
# 计算验证集的准确率
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据移动到GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Epoch 1, Train Loss: 1.3520895305432772
Epoch [1/40], Validation Accuracy: 56.34%
Epoch 2, Train Loss: 1.1349403094827084
Epoch [2/40], Validation Accuracy: 64.89%
Epoch 3, Train Loss: 0.9625815153121948
Epoch [3/40], Validation Accuracy: 66.17%
Epoch 4, Train Loss: 0.8833337227503458
Epoch [4/40], Validation Accuracy: 66.74%
Epoch 5, Train Loss: 0.8212433541030214
Epoch [5/40], Validation Accuracy: 68.80%
Epoch 6, Train Loss: 0.775716503461202
Epoch [6/40], Validation Accuracy: 68.95%
Epoch 7, Train Loss: 0.7365252678854424
Epoch [7/40], Validation Accuracy: 69.16%
Epoch 8, Train Loss: 0.7170723237489399
Epoch [8/40], Validation Accuracy: 71.94%
Epoch 9, Train Loss: 0.6886676183918066
Epoch [9/40], Validation Accuracy: 70.09%
Epoch 10, Train Loss: 0.6675241328122323
Epoch [10/40], Validation Accuracy: 72.51%
Epoch 11, Train Loss: 0.6403116991645411
Epoch [11/40], Validation Accuracy: 73.22%
Epoch 12, Train Loss: 0.633016062933102
Epoch [12/40], Validation Accuracy: 72.65%
Epoch 13, Tr

### 5.保存模型

In [13]:
# 保存模型权重
torch.save(model.state_dict(), 'lenet5_brain_weights.pth')
# 保存模型结构和权重
torch.save(model, 'lenet5_brain_model.pth')
accuracies_list = model_accuracies['lennet_5']

# 创建一个包含epoch编号和对应准确率的字典
data = {'Epoch': list(range(1, len(accuracies_list) + 1)),
        'Accuracy': accuracies_list}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 现在你可以将DataFrame保存为Excel文件
df.to_excel('lennet_5_brain_accuracies.xlsx', index=False)
# 清理内存
del model
del inputs
del labels
torch.cuda.empty_cache()

### （示例）下次调用时可加载模型

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 实例化模型
model = LeNet5(num_classes=4).to(device)

# 加载模型权重
model.load_state_dict(torch.load('lenet5_brain_weights.pth'))

# 设置为评估模式
model.eval()

Using device: cuda


LeNet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)

 ## 评估模型

In [15]:
# 确保模型处于评估模式
model.eval()

# 初始化用于存储预测和真实标签的列表
all_preds = []
all_labels = []

# 不计算梯度进行预测
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 将数据移动到GPU，如果可用
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测概率最高的类别
        _, predicted = torch.max(outputs.data, 1)

        # 将预测结果和真实标签添加到列表中
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 计算准确率、精确率、召回率和F1分数
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

# 将结果保存到CSV文件
results = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# 写入CSV文件
with open('lennet5_brain评估.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Value'])
    for metric, value in results.items():
        writer.writerow([metric, value])

print("Results have been saved to lennet5_brain评估.csv")

Evaluating: 100%|██████████| 15/15 [00:00<00:00, 20.26it/s]

Results have been saved to lennet5_brain评估.csv
